In [ ]:
import os
import sys

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import numpy as np

from Intra import Intra
from Utilities import show_results

In [ ]:
V_intra = Intra(use_D3=True)

RT = lambda T: (Boltzmann * Avogadro) * T / 1000
RMSE = lambda x, y, n=1: np.round(np.sqrt(np.mean(np.square(np.subtract(x, y)))), n)
MAE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y))), n) 
MARE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y)) / x), n) 
ME =  lambda x, y, n=1: np.round(np.mean(np.subtract(x, y)), n)
STD =  lambda x, y, n=1: np.round(np.std(np.subtract(x, y)), n)
MAE =  lambda x, y, n=1: np.round(np.mean(np.abs(np.subtract(x, y))), n)
from scipy.stats import pearsonr
def print_out_results(target=None, prediction=None, stds=None, property_name='', unit='', n_digits=1):
    mae = MAE(target, prediction, n_digits)
    mare = MARE(target, prediction, n_digits)
    rmse = RMSE(target, prediction, n_digits)
    me = ME(target, prediction, n_digits)
    max_abs = np.round(np.amax(np.abs(prediction - target)), n_digits)
    r = np.round(pearsonr(np.array(target).squeeze().flatten(), np.array(prediction).squeeze().flatten()).statistic, 2)
    print(f'{property_name} [{unit}]& {rmse:{4}.{n_digits}f}\\')
    print(f'{property_name} [{unit}]& {len(target)} & {mae:{4}.{n_digits}f} & {me:{4}.{n_digits}f} & {rmse:{4}.{n_digits}f} & {r:{4}.{2}f}\\')

In [ ]:
system_key = 'glucose'
data = np.load(f'../../../data/test_sets/{system_key}.npy', allow_pickle=True).item()
references, coordinates = [], []
pred_alpha, ref_alpha = [], []
pred_beta, ref_beta = [], []
pred_oc, ref_oc = [], []
for index in data:    
    anomer = data[index]['anomer']
    coords = data[index]['coordinates'][None]
    ref = data[index]['ref']
    references.append(ref)
    elements = data[index]['elements']
    V_intra.initialize(coords, elements)
    V_total, _ = V_intra(coords)
    if anomer == 'alpha':        
        ref_alpha.append(data[index]['ref'])
        pred_alpha.append(V_total)        
    if anomer == 'beta':        
        ref_beta.append(data[index]['ref'])
        pred_beta.append(V_total)
    if anomer == 'oc':        
        ref_oc.append(data[index]['ref'])
        pred_oc.append(V_total)

ref_alpha, pred_alpha = np.hstack(ref_alpha), np.hstack(pred_alpha)[0]
ref_beta, pred_beta = np.hstack(ref_beta), np.hstack(pred_beta)[0]
ref_oc, pred_oc = np.hstack(ref_oc), np.hstack(pred_oc)[0]
pred_alpha -= pred_alpha[np.argmin(ref_alpha)]
pred_beta -= pred_beta[np.argmin(ref_beta)]                               
pred_oc -= pred_oc[np.argmin(ref_oc)]
ref_alpha -= ref_alpha[np.argmin(ref_alpha)]
ref_beta -= ref_beta[np.argmin(ref_beta)]
ref_oc -= ref_oc[np.argmin(ref_oc)]
references = np.hstack((ref_alpha, ref_beta, ref_oc))
predictions = np.hstack((pred_alpha, pred_beta, pred_oc))

In [ ]:
print_out_results(references, predictions, property_name='Glucose', unit='kJ/mol')

In [ ]:
show_results(ref_alpha, pred_alpha, show_plot=True, show_chem_acc=True)

In [ ]:
show_results(ref_oc, pred_oc, show_plot=True, show_chem_acc=True)

In [ ]:
show_results(ref_beta, pred_beta, show_plot=True, show_chem_acc=True)

In [ ]:
show_results(references, predictions, show_plot=True, show_chem_acc=True)

In [ ]:
system_key = 'maltose'
data = np.load(f'../../../data/test_sets/{system_key}.npy', allow_pickle=True).item()
references, coordinates = [], []
for index in data:    
    coordinates.append(data[index]['coordinates'])
    references.append(data[index]['ref'])
elements = data[index]['elements']
references = np.hstack(references)[:, None]
coordinates = np.array(coordinates)
V_intra.initialize(coordinates, elements)
predictions, _ = V_intra(coordinates)
predictions -= predictions[np.argmin(references)]
references -= references[np.argmin(references)]

In [ ]:
show_results(references, predictions, show_plot=True, show_chem_acc=True)

In [ ]:
print_out_results(references, predictions, property_name='Maltose', unit='kJ/mol')

In [ ]:
for key in ['SCONF',  'PCONF', 'ACONF', 'CYCONF']: 
    data = np.load(f'../../../data/test_sets/data_{key}.npy', allow_pickle=True, encoding='bytes').item()
    predictions, references = [], []
    for system_name in data:        
        coords_1 = data[system_name][b'coordinates_1'][None]     
        coords_2 = data[system_name][b'coordinates_2'][None]         
        elements_1 = data[system_name][b'elements_1'].astype(str)        
        elements_2 = data[system_name][b'elements_2'].astype(str)   
        V_intra.initialize(coords_1, elements_1)
        V1, _ = V_intra(coords_1)
        V_intra.initialize(coords_2, elements_2)
        V2, _ = V_intra(coords_2)
        predictions.append(V2 - V1)
        references.append(data[system_name][b'ref_energy'] )
    predictions, references = np.hstack(predictions)[0], np.hstack(references)
    show_results(references, predictions, dataset_name=key, show_plot=False, show_chem_acc=True)
    #plt.show()
    print_out_results(references, predictions, property_name=key, unit='kJ/mol')